In [13]:
%matplotlib widget
from typing import Tuple, List, Optional
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from os.path import abspath, dirname, join

from mpl_toolkits.mplot3d import Axes3D
# from ipywidgets import widgets


def load_files(specific_path: str) -> Tuple[np.ndarray, str]:
    """
    Loads ".csv" files from the specified subfolder of the Resources in the form of a numpy array. 
    
    Args:
        specific_path (str): The subfolder path where the data to be loaded is found
    Returns: 
        data_array (np.ndarray): An array of shape <bun samples x num. time samples x num attributes>. In this case,
                                      the attributes are: gain A, gain B, velocity, acceleration.
    """
    print("Reading csv volume controller swipe files...")
    all_rsc_path = join(os.getcwd(), "Resources")
    data_dirpath = os.path.abspath(os.path.join(all_rsc_path, specific_path))
    print(f"Data directory: {data_dirpath}\n")
    
    data_ls = []
    labels = []
    for filename in os.listdir(data_dirpath):
        if filename.endswith(".csv"):
            # split the filename into the name part and the extension part
            name, extension = os.path.splitext(filename)
            filepath = os.path.join(data_dirpath, filename)
            filedata = _get_file_data(filepath)
            data_ls.append(filedata)
    
            # extract swipe sample number from the filename
            swipe_num_str = name[name.find("0"): name.find("_velocity")]
            labels.append(swipe_num_str)
            
    data_array = np.stack(data_ls, axis=0)
    print(f"\nData array shape: {data_array.shape}\n")
    return data_array, labels

def _get_file_data(filepath: str) -> np.ndarray:
    """
    Obtain the data in the give file, potentially filtering out some rows and columns.

    Args:
        filepath (str): the path to the file whose data is to be extracted
    Returns:
        file_lines (np.ndarray): the extracted data from the given file
    """
    with open(filepath, encoding='ascii') as f:
        # get the data in each file by first stripping and splitting the lines and
        # then creating a numpy array out of these values
        file_lines = []
        print("Filename: ", filepath)
        for line in f:
            line = line.strip(' \t\n\r')
            line = re.split('\t|,', line)
            file_lines.append(line)
        file_lines = np.asarray(file_lines)

        # keep info only from the relevant columns and rows - in this case get everything
        file_lines = (file_lines[:]).astype(np.float32)
        return file_lines

specific_path = "volume_slider_swipes\p001"
data_array, labels = load_files (specific_path)

Reading csv volume controller swipe files...
Data directory: C:\GitHub\Knitted_Sensor_Gestures\Resources\volume_slider_swipes\p001

Filename:  C:\GitHub\Knitted_Sensor_Gestures\Resources\volume_slider_swipes\p001\volumeslider_swipe000_velocity_acceleration_data.csv
Filename:  C:\GitHub\Knitted_Sensor_Gestures\Resources\volume_slider_swipes\p001\volumeslider_swipe001_velocity_acceleration_data.csv
Filename:  C:\GitHub\Knitted_Sensor_Gestures\Resources\volume_slider_swipes\p001\volumeslider_swipe002_velocity_acceleration_data.csv
Filename:  C:\GitHub\Knitted_Sensor_Gestures\Resources\volume_slider_swipes\p001\volumeslider_swipe003_velocity_acceleration_data.csv
Filename:  C:\GitHub\Knitted_Sensor_Gestures\Resources\volume_slider_swipes\p001\volumeslider_swipe004_velocity_acceleration_data.csv
Filename:  C:\GitHub\Knitted_Sensor_Gestures\Resources\volume_slider_swipes\p001\volumeslider_swipe005_velocity_acceleration_data.csv
Filename:  C:\GitHub\Knitted_Sensor_Gestures\Resources\volume_sl

In [14]:
def define_plot_env(fig, title: str, ax_labels: List[str]) -> Axes3D: 
    """
    Creates the axis on which to plot. 
    
    Args:
        fig: The figure object that will contain this axis
        title: The title of the plot
        ax_labels: The labels of each of the three axis 

    Returns:

    """
    sns.set(context='notebook', style='darkgrid', palette='pastel', font='sans-serif', font_scale=1,
            color_codes=True,
            rc=None)
    # fig = plt.figure(figsize=(10, 9))
    ax = fig.add_subplot(111, projection='3d')

    plt.title(title, fontsize=12)
    sns.axes_style("white")
    
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax.yaxis.set_major_locator(plt.MaxNLocator(5))
    ax.zaxis.set_major_locator(plt.MaxNLocator(5))
    
    ax.set_xlabel(ax_labels[0], fontsize=10, labelpad=8)
    ax.set_ylabel(ax_labels[1], fontsize=10, labelpad=8)
    ax.set_zlabel(ax_labels[2], fontsize=10, labelpad=8)
    return ax


In [15]:
#TODO: pass as a parameter the number of sample to plot, or plot all in one
def plot_gains_time(all_data: np.ndarray, labels: List[str], filter: Optional[List[int]] = None) -> None:
    """
    Plots the gain values over time for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
    # prepare axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain Trajectory over Time"
    ax_labels = ["Time", "Gain A", "Gain B"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
        
    x = np.arange(0, all_data.shape[1]) 
    for i in idx:
        sample = all_data[i, :, :]

        y = sample[:, 0] # gain A
        z = sample[:, 1] # gain B
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])

    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_gains_time = [5, 8, 11, 18, 25] # change this for samples to plot, or set to None to plot all samples
filter_gains_time = None
plot_gains_time(data_array, labels, filter_gains_time)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
def plot_gains_vel(all_data: np.ndarray, labels: List[str], filter: Optional[List[int]] = None) -> None:
    """
    Plots the gain values with the related particle velocity over time for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
    # define plotting axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain Trajectory vs Velocity"
    ax_labels = ["Gain A", "Gain B", "Velocity"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine the number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
    
    for i in idx: 
        sample = all_data[i, :, :]
        x = sample[:, 0]
        y = sample[:, 1]
        z = sample[:, 2]
    
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_gains_vel = [5, 8, 11, 17, 25]
filter_gains_vel = None
plot_gains_vel(data_array, labels, filter_gains_vel)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
def plot_gains_acc(all_data: np.ndarray, labels: List[str], filter: Optional[List[int]] = None) -> None:
    """
    Plots the gain values with the related particle acceleration for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
      
    # define plotting axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain Trajectory vs Acceleration"
    ax_labels = ["Gain A", "Gain B", "Acceleration"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine the number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
    
    for i in idx: 
        sample = all_data[i, :, :]
        x = sample[:, 0]
        y = sample[:, 1]
        z = sample[:, 3]
    
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_gains_acc = [5, 8, 11, 17, 25]
filter_gains_acc = None
plot_gains_acc(data_array, labels, filter_gains_acc)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
def plot_time_gainA_vel(all_data: np.ndarray, labels: List[str], filter: Optional[List[int]] = None) -> None:
    """
    Plots the values of gain A and the particle motion velocity over time for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                  number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
    
    # define plotting axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain A Value and Velocity over Time"
    ax_labels = ["Time", "Gain A", "Velocity"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine the number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
    
    x = np.arange(0, all_data.shape[1])
    for i in idx: 
        sample = all_data[i, :, :]
        y = sample[:, 0]
        z = sample[:, 2]
    
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_gains_acc = [5, 8, 11, 17, 25]
filter_time_gainA_vel = None
plot_time_gainA_vel(data_array, labels, filter_time_gainA_vel)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:

def plot_time_gainB_vel(all_data: np.ndarray, labels: List[str], filter: Optional[List[int]] = None) -> None:
    """
    Plots the values of gain B and the particle motion velocity over time for several swipe samples.
    
    Args:
        all_data: All swipe data in numpy array format of shape <number of samples/instances x 
                               number of time steps x number of attributes (gain A, gain B, velocity, acceleration))> 
        labels: The ids of the swipe samples
        filter: If this argument is set, it plots only the swipe lines specified. By default it is set to None, so the 
                function plots all the swipe data. Otherwise, a list of int elements to expected, corresponding to the identities 
                of swipes to be plotted.  
    """
    
    # define plotting axis
    fig = plt.figure(figsize=(10, 9))
    title = "Gain B Value and Velocity over Time"
    ax_labels = ["Time", "Gain B", "Velocity"]
    ax = define_plot_env(fig, title, ax_labels)
    
    # determine the number of samples to plot
    if filter is None:
        idx = range(0, all_data.shape[0])
    else:
        idx = filter
    
    x = np.arange(0, all_data.shape[1])
    for i in idx: 
        sample = all_data[i, :, :]
        y = sample[:, 1]
        z = sample[:, 2]
    
        line, = ax.plot(x, y, z)
        line.set_label(labels[i])
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fancybox=True, shadow=True)  
    plt.show()

# filter_gains_acc = [5, 8, 11, 17, 25]
filter_time_gainB_vel = None
plot_time_gainB_vel(data_array, labels, filter_time_gainB_vel)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …